In [1]:
%pip install pi-heaan
import piheaan as heaan
from piheaan.math import sort
from piheaan.math import approx # for piheaan math function
import math
import numpy as np
import pandas as pd
import os

Note: you may need to restart the kernel to use updated packages.


In [4]:
# set parameter
params = heaan.ParameterPreset.FGb
context = heaan.make_context(params) # context has paramter information
heaan.make_bootstrappable(context) # make parameter bootstrapable

# create and save keys
key_file_path = "./keys"
sk = heaan.SecretKey(context) # create secret key
os.makedirs(key_file_path, mode=0o775, exist_ok=True)
sk.save(key_file_path+"/secretkey.bin") # save secret key


In [5]:
key_generator = heaan.KeyGenerator(context, sk) # create public key
key_generator.gen_common_keys()
key_generator.save(key_file_path+"/") # save public key

In [6]:
# load secret key and public key
# When a key is created, it can be used again to save a new key without creating a new one
key_file_path = "./keys"

sk = heaan.SecretKey(context,key_file_path+"/secretkey.bin") # load secret key
pk = heaan.KeyPack(context, key_file_path+"/") # load public key
pk.load_enc_key()
pk.load_mult_key()

eval = heaan.HomEvaluator(context,pk) # to load piheaan basic function
dec = heaan.Decryptor(context) # for decrypt
enc = heaan.Encryptor(context) # for encrypt

In [7]:
log_slots = 15
num_slots = 2**log_slots

In [50]:
card_num = [4,2,6,3,9,8,2,6,4,0,2,6,9,2,9,9] + [0]*(num_slots-16)
print(len(card_num))
print(card_num)

32768
[4, 8, 5, 4, 7, 9, 7, 6, 1, 0, 5, 1, 1, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [51]:
card_num_msg = heaan.Message(log_slots)
for i in range(num_slots):
    card_num_msg[i] = card_num[i]
card_num_ctxt = heaan.Ciphertext(context)

In [52]:
enc.encrypt(card_num_msg, pk, card_num_ctxt)

In [53]:
# reversed Card num
rev_card_num_msg = heaan.Message(log_slots)

for i in range(16):
    # print(card_num[i])
    rev_card_num_msg[15-i] = card_num[i]
# print(rev_card_num_msg)

for i in range(num_slots-16):
    rev_card_num_msg[i+15] = card_num[i+15]
rev_card_num_ctxt = heaan.Ciphertext(context)

enc.encrypt(rev_card_num_msg, pk, rev_card_num_ctxt)

# print(len(rev_card_num_msg))

1. 홀수 자리에만 *2
2. 만약 1에서 그 숫자가 >=10 이면 각 자리 수 더하기
3. 홀수자리 합 + 짝수 자리 합 % 10 == 0
=> valid

In [54]:
card_num_copy = heaan.Ciphertext(context)
card_num_copy_msg = heaan.Message(log_slots)

card_num_copy = card_num_ctxt

dec.decrypt(card_num_copy, sk, card_num_copy_msg)

print("cp: ", card_num_copy_msg)

cp:  [ (4.000000+0.000000j), (8.000000+0.000000j), (5.000000+0.000000j), (4.000000+0.000000j), (7.000000+0.000000j), ..., (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j) ]


1. 홀수 자리에 *2

In [55]:
double_odd = heaan.Ciphertext(context)
double_odd_msg = heaan.Message(log_slots)

eval.mult(card_num_ctxt, 2, double_odd)

dec.decrypt(double_odd, sk, double_odd_msg)

print("double odd: ", double_odd_msg)

double odd:  [ (8.000000+0.000000j), (16.000000+0.000000j), (10.000000+0.000000j), (8.000000+0.000000j), (14.000000+0.000000j), ..., (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j) ]


In [56]:
def division(divided):
    # print("division")

    # 몫을 세기 위함
    cnt = [0] * num_slots

    cnt_msg = heaan.Message(log_slots)
    for i in range(num_slots):
        cnt_msg[i] = cnt[i]
    cnt_ctxt = heaan.Ciphertext(context)

    enc.encrypt(cnt_msg, pk, cnt_ctxt)

    # 1을 더해주기 위함
    sig = [1] * num_slots

    sig_msg = heaan.Message(log_slots)
    for i in range(num_slots):
        sig_msg[i] = sig[i]
    sig_ctxt = heaan.Ciphertext(context)

    enc.encrypt(sig_msg, pk, sig_ctxt)

    # divider(10) encryption
    ten = [10] * num_slots

    ten_msg = heaan.Message(log_slots)
    for i in range(num_slots):
        ten_msg[i] = ten[i]
    ten_ctxt = heaan.Ciphertext(context)

    enc.encrypt(ten_msg, pk, ten_ctxt)

    # 나눗셈 연산을 위한 빼기
    # 나머지 선언
    remain = heaan.Ciphertext(context)
    remain_msg = heaan.Message(log_slots)

    # 나눠지는 수 선언
    divided_list = heaan.Ciphertext(context)
    divided_msg = heaan.Message(log_slots)
    divided_list = divided



    eval.sub(divided_list, ten_ctxt, remain)
    eval.add(cnt_ctxt, sig_ctxt, cnt_ctxt)
    dec.decrypt(remain, sk, remain_msg)
    dec.decrypt(cnt_ctxt, sk, cnt_msg)
    dec.decrypt(divided_list, sk, divided_msg)

    print("remain: ", remain_msg)
    print("cnt: ", cnt_msg)
    print()

    


    while True:
        if remain_msg[0].real <= 0:
            break
        eval.sub(remain, ten_ctxt, remain)
        dec.decrypt(remain, sk, remain_msg)

        # print("remain: ", remain_msg)
        eval.add(cnt_ctxt, sig_ctxt, cnt_ctxt)

        dec.decrypt(cnt_ctxt, sk, cnt_msg)
        # print("cnt: ", cnt_msg)

    return cnt_ctxt, remain



In [57]:
cnt, remain = division(double_odd)
adding_r_c = heaan.Ciphertext(context)
adding_r_c_msg = heaan.Message(log_slots)

eval.add(cnt, remain, adding_r_c)

dec.decrypt(adding_r_c, sk, adding_r_c_msg)

print("add: ", adding_r_c_msg)

remain:  [ (-2.000000+0.000000j), (6.000000+0.000000j), (0.000000+0.000000j), (-2.000000+0.000000j), (4.000000+0.000000j), ..., (-10.000000+0.000000j), (-10.000000+0.000000j), (-10.000000+0.000000j), (-10.000000+0.000000j), (-10.000000+0.000000j) ]
cnt:  [ (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), ..., (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j) ]

add:  [ (-1.000000+0.000000j), (7.000000+0.000000j), (1.000000+0.000000j), (-1.000000+0.000000j), (5.000000+0.000000j), ..., (-9.000000+0.000000j), (-9.000000+0.000000j), (-9.000000+0.000000j), (-9.000000+0.000000j), (-9.000000+0.000000j) ]
